# Model diagnostics and performance

Load The Data

In [2]:
# Load pickled data
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load up the augmented training datasign_names = pd.read_csv('signnames.csv')
validation_file='../data/valid.p'
testing_file = '../data/test.p'

with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']
sign_names = pd.read_csv('signnames.csv')

In [3]:
# Helper functions
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.contrib.layers import flatten


In [4]:

# Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
mu = 0
sigma = 0.1

def flatten_conv(x, input_shape, output_len):
    if input_shape[0] * input_shape[1] * input_shape[2] - output_len:
        raise Exception('Input and output shape mismatch while flattening')
    return flatten(x)

def fc_relu(x, input_len, output_len):
    _fc = fc(x, input_len, output_len)
    return  tf.nn.relu(_fc)

def fc(x, input_len, output_len):
    fc_W  = tf.Variable(tf.truncated_normal(shape=(input_len, output_len), mean = mu, stddev = sigma))
    fc_b  = tf.Variable(tf.zeros(output_len))
    _fc    = tf.matmul(x, fc_W) + fc_b
    return _fc

def max_pool(x, input_shape, output_shape, kernel_width=2, stride=2):
    input_width = input_shape[0]
    output_width = output_shape[0]
    if input_width/2 == output_width:
        return tf.nn.max_pool(x, ksize=[1, kernel_width, kernel_width, 1], strides=[1, stride, stride, 1], padding='VALID')
    elif input_width == output_width:
        return tf.nn.max_pool(x, ksize=[1, kernel_width, kernel_width, 1], strides=[1, 1, 1, 1], padding='SAME')

def conv2d_relu(x, input_shape, output_shape, kernel_width = 5, strides=[1, 1, 1, 1]):
    """
    Define a standard convolution layer with relu activation
    """
    input_width = input_shape[0]
    input_depth = input_shape[2]
    output_width = output_shape[0]
    kernel_depth = output_shape[2]
    if output_width % 2:
        raise Exception('Output shape is odd. Not supported at the moment')
    elif  input_shape[0]!= input_shape[1] or output_shape[0]!= output_shape[1]:
        raise Exception('Only square images supported')
    if input_shape[0] == output_shape[0]:
        padding = 'SAME'
    elif input_shape[0] > output_shape[0]:
        padding = 'VALID'
        if input_width - output_width + 1 - kernel_width:
            raise Exception(
                'Kernel width of {} does not support {} - {} convolution'.format(
                    kernel_width,input_width,output_width))
    else:
        raise Exception('Output size is larger than input')

    conv_W = tf.Variable(tf.truncated_normal(shape=(kernel_width, kernel_width, input_depth, kernel_depth), mean = mu, stddev = sigma))
    conv_b = tf.Variable(tf.zeros(kernel_depth))
    conv = tf.nn.conv2d(x, conv_W, strides=strides, padding=padding) + conv_b
    return tf.nn.relu(conv)

# Model training

### Model Architecture

In [5]:
def KNet(x):        
    # Convolutional layer #1
    h = conv2d_relu(x, input_shape=(32,32,3),output_shape=(32,32,5), kernel_width=3)
    h = max_pool(h, input_shape = (32,32,5), output_shape=(32,32,5))
    #h = tf.nn.dropout(h, 0.5)

    # Convolutional layer #2
    h = conv2d_relu(h, input_shape=(32,32,5),output_shape=(32,32,10), kernel_width=3)
    h = max_pool(h, input_shape = (32,32,10), output_shape=(32,32,10))

    # Convolutional layer #3
    h = conv2d_relu(h, input_shape=(32,32,10),output_shape=(32,32,20), kernel_width=3)
    h = max_pool(h, input_shape = (32,32,20), output_shape=(32,32,20))
    h = tf.nn.dropout(h, 0.5)

    # Convolutional layer #4
    h = conv2d_relu(h, input_shape=(32,32,20), output_shape=(28,28,25), kernel_width=5)
    h = max_pool(h, input_shape = (28,28,25), output_shape=(14,14,25))

    # Convolutional layer #5
    h = conv2d_relu(h, input_shape=(14,14,25), output_shape=(10,10,30), kernel_width=5)
    h = max_pool(h, input_shape = (10,10,30), output_shape=(5,5,30))
    #h = tf.nn.dropout(h, 0.5)

    # Flatten convolution
    h   = flatten_conv(h, input_shape=(5,5,30), output_len=750)
    
    # Fully connected layers
    h = fc_relu(h, input_len=750, output_len=43)
    h = fc(h, input_len=43, output_len=43)

    return h

### Train, Validate and Test the Model

In [6]:
BATCH_SIZE = 128
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
is_training = tf.placeholder(tf.bool, True)
one_hot_y = tf.one_hot(y, 43)
rate = tf.placeholder(tf.float32, [])
logits = KNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [8]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

Test Accuracy = 0.941


---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

Test Accuracy = 0.944
